In [1]:
from lib_maputils import MapUtils
from lib_mapvis import MapVis
import json
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv


In [2]:
def read_json_map(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

def read_map(data, out):
  for feature in data['features']:
    coords = feature['geometry']['coordinates']
    out.append(coords)

In [3]:
%timeit
filename = 'world_map_latlon.json'

map_data = read_json_map(filename)
out = [] 
read_map(map_data, out)
mercator_all = []
for polygon in out:
  mercator_all += polygon

In [4]:
def lerp(v, from_min, from_max, to_min, to_max):
    return (v - from_min) / (from_max - from_min) * (to_max - to_min) + to_min

In [13]:
from tqdm import tqdm 
min_mercator = np.array((-20037508.34, -20037508.34))
max_mercator = np.array((20037508.34, 20037508.34))

height , width = 1500, 1500
img = np.ones((width, height, 3), dtype=np.uint8)*255

for feature in tqdm(map_data['features']):
  coords = feature['geometry']['coordinates']
  coords = np.array(coords)
  feature_type = feature['properties']['type']

  mercator = []
  for latlon in coords:
    mercator.append(MapUtils.latlon_to_mercator(*latlon[::-1]))
  mercator = np.array(mercator)

  mercator2pixel = lerp(mercator, min_mercator, max_mercator, 0, height).reshape((-1, 1, 2)).astype(np.int32)
  if feature_type == 'state':
    img = cv.polylines(img,[mercator2pixel],False,(50,50,50), 1)
  elif feature_type == 'country':
    img = cv.polylines(img,[mercator2pixel],False,(0,150,150), 1)
  elif feature_type == 'river':
    img = cv.polylines(img,[mercator2pixel],False,(120,30,30), 1)
  elif feature_type == 'coastline':
    img = cv.polylines(img,[mercator2pixel],False,(0,0,0), 1)

cv.imwrite("world_map_topology.png",img)

100%|██████████| 25359/25359 [00:01<00:00, 21472.51it/s]


True